In [6]:
search_for = 108
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016303300857543945
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 525
all cases: 602676921
type: [1, 1, 1, 1, 108] 108
cases of this type: 136048896
10000 85.79323525055563
20000 86.819447989842
30000 84.04553915801614
40000 264.92585808456744
50000 465.95855984645465
60000 469.0619317712904
70000 444.192447778384
80000 434.88353800906066
90000 419.6272807187549
100000 431.3489795401646
110000 454.5046905269225
120000 424.2403629394146
130000 416.1248721439195
140000 452.04003785404535
150000 441.9896995622763
160000 411.561594757389
170000 419.96343268644466
180000 438.8050845164192
190000 419.848997160363
200000 424.0333135652927
210000 415.79761439332003
220000 401.08925827710243
230000 417.2484486282519
240000 412.75510854397464
250000 428.7970367363423
260000 417.22899949474106
270000 428.6238560807526
280000 413.22773296940204
290000 405.7782572996892
300000 433.48669826030675
310000 424.8022156774679

3080000 392.15515960361387
3090000 439.35619048278454
3100000 390.52863493208923
3110000 419.6198839366685
3120000 518.330192226804
3130000 328.08838118265294
3140000 53.289320676006575
3150000 53.81142739955435
3160000 53.80184250396243
3170000 53.30507670333051
3180000 53.8927425202874
3190000 349.74901782803613
3200000 521.6002525398632
3210000 435.44130445258963
3220000 401.70067620731953
3230000 451.36308678250583
3240000 384.7141018707221
3250000 379.22857170052237
3260000 407.4325916289546
3270000 394.189219857604
3280000 382.7082263680036
3290000 411.7793167680662
3300000 416.5492636729001
3310000 391.66792851652576
3320000 382.27698401156033
3330000 407.1857756500522
3340000 400.6882750768245
3350000 375.9807774990935
3360000 386.1019020972371
3370000 398.0327514933217
3380000 380.60388571565437
3390000 404.5476003423717
3400000 398.80289077471235
3410000 396.9580125045527
3420000 417.10440279844283
3430000 380.2465390465118
3440000 409.9918703433443
3450000 419.5063844317412


6190000 366.4679607701954
6200000 408.2952936987287
6210000 415.14020267330596
6220000 404.98328375939866
6230000 427.8018006374327
6240000 130.561020663889
6250000 129.56269423805085
6260000 129.48853475549743
6270000 128.80723881054197
6280000 138.16343282924723
6290000 303.5063913228672
6300000 549.046607048995
6310000 465.0137866942691
6320000 414.9275543492873
6330000 489.00531218735904
6340000 247.60696171234432
6350000 106.27688656386901
6360000 103.78096680355712
6370000 103.96139442894759
6380000 104.48062266466415
6390000 109.04368266019516
6400000 495.3979449449778
6410000 472.4488927605973
6420000 455.2499433847582
6430000 372.8160087220204
6440000 439.08607019487613
6450000 414.813088465095
6460000 475.4351145433915
6470000 372.2514441327989
6480000 387.78036580664127
6490000 461.0061939945249
6500000 374.87362054590733
6510000 438.3717419808759
6520000 374.77497800565857
6530000 376.13869483093134
6540000 420.8976279700432
6550000 382.3871499204663
6560000 449.92940815039

9300000 411.7069061201459
9310000 394.3770149951064
9320000 392.8922041112228
9330000 416.6645012406582
9340000 376.38077381031064
9350000 356.18270554557824
9360000 411.9555826191444
9370000 374.8919708216436
9380000 383.6387328924843
9390000 372.65980812374346
9400000 341.4529211049475
9410000 381.7237475943399
9420000 396.03289751049834
9430000 231.05844024666803
9440000 56.17177616105852
9450000 69.34656537044052
9460000 67.45385108892003
9470000 51.57496376413913
9480000 51.53041236826855
9490000 337.0480456687046
9500000 453.4316581530937
9510000 365.8706819647622
9520000 314.2351491704206
9530000 397.8438117461067
9540000 373.97608440630205
9550000 360.7680250146743
9560000 384.29675370249385
9570000 389.65010975542026
9580000 388.3397005272566
9590000 381.09585575796973
9600000 398.4639606979786
9610000 418.0899306167569
9620000 380.8321291084258
9630000 388.9109005877588
9640000 401.8060730205657
9650000 371.51483761212694
9660000 392.06704896951516
9670000 370.323724395394
96

12320000 399.9363155087322
12330000 373.0823578242914
12340000 408.2971560074743
12350000 391.5567376928785
12360000 396.3656844680371
12370000 409.46540312084403
12380000 398.84771966089204
12390000 395.65940535596644
12400000 377.87775516739924
12410000 372.428589667976
12420000 438.7054264671131
12430000 371.6307400804039
12440000 401.49869894403804
12450000 395.77135083219224
12460000 379.9231579728205
12470000 428.62959497447935
12480000 129.26225750025716
12490000 128.78594376359177
12500000 127.03754945287417
12510000 127.60345355569166
12520000 152.66348814843315
12530000 241.75740771777467
12540000 459.7146941033066
12550000 476.3790833100335
12560000 388.1315271011614
12570000 361.69604054038626
12580000 396.70818926431235
12590000 437.5103408387919
12600000 449.61264435837893
12610000 398.4941356164841
12620000 444.988879022614
12630000 415.1059255706764
12640000 376.02331695048457
12650000 415.5134651453875
12660000 401.261581559633
12670000 409.58050648322336
12680000 413.

...

595890000 2.0878598050199546
595900000 2.5951773829907974
595910000 2.965815903145221
595920000 2.0482372597818967
595930000 1.326406688400613
595940000 1.5984001625727418
595950000 3.0689702282576765
595960000 1.8683483347923888
595970000 2.032615569695433
595980000 1.6910187512706718
595990000 1.7514626287461916
596000000 2.5723203327747717
596010000 1.93411134530282
596020000 1.3530680254726277
596030000 2.305443463058094
596040000 1.704913882700821
596050000 1.9064897544275854
596060000 2.5845041735443672
596070000 1.2543677463340164
596080000 2.3692043937800196
596090000 2.236744838267598
596100000 1.9445907797560693
596110000 2.18103468918605
596120000 1.836094933504005
596130000 2.2238126171090404
596140000 1.9902789506892962
596150000 2.546666071376423
596160000 1.4332207709316414
596170000 2.664969037370424
596180000 1.64960777197106
596190000 1.7938884585724475
596200000 2.5885661044899355
596210000 1.563458253801253
596220000 2.729643657369693
596230000 1.5050790219926238
59

598590000 0.6618858371703029
598600000 0.6879606058705051
598610000 0.6731480875974297
598620000 0.6477157884277106
598630000 0.6257520448387796
598640000 0.6909185568518573
598650000 0.6842062814154625
598660000 0.7369600909095673
598670000 0.6615428626433545
598680000 0.6374980844754974
598690000 0.6645402757062382
best so far: 0
type: [3, 1, 1, 18, 2] 108
cases of this type: 93312
598700000 0.6657952138872941
598710000 0.25439263380843397
598720000 0.26759645020459755
598730000 0.2563113333375322
598740000 0.2581086204686761
598750000 0.2353056823899547
598760000 0.24893131049468784
598770000 0.24951154831395547
598780000 0.254130676116036
598790000 0.25382114523259797
best so far: 0
type: [3, 1, 1, 36, 1] 108
cases of this type: 46656
598800000 0.26393364925396445
598810000 0.2465806044941611
598820000 0.24704590105919041
598830000 0.25747150279957054
best so far: 0
type: [3, 1, 2, 1, 18] 108
cases of this type: 419904
598840000 0.25440362248013415
598850000 1.0074805459248954
5988

600700000 0.34935095914326114
best so far: 0
type: [3, 2, 1, 2, 9] 108
cases of this type: 104976
600710000 0.21368239321649735
600720000 0.2536546401692231
600730000 0.2637053413215545
600740000 0.20462160811925598
600750000 0.13426073822160564
600760000 0.14515564181168875
600770000 0.13500889500004715
600780000 0.14695908906900884
600790000 0.19256607352850835
600800000 0.24628824914462039
best so far: 0
type: [3, 2, 1, 3, 6] 108
cases of this type: 69984
600810000 0.2103428938279549
600820000 0.18565850951090787
600830000 0.1789071124401556
600840000 0.16639307270046078
600850000 0.17547469282743666
600860000 0.1837491472551889
600870000 0.1742809009037018
best so far: 0
type: [3, 2, 1, 6, 3] 108
cases of this type: 34992
600880000 0.16175410558599898
600890000 0.172198218748841
600900000 0.16288927109648785
600910000 0.16679209161833922
best so far: 0
type: [3, 2, 1, 9, 2] 108
cases of this type: 23328
600920000 0.12046747328895992
600930000 0.11697822132323185
best so far: 0
type

602020000 0.08596939215925667
602030000 0.05544715911649333
602040000 0.08608887604886294
602050000 0.05540677081830635
best so far: 0
type: [4, 1, 1, 9, 3] 108
cases of this type: 59049
602060000 0.08147749085904493
602070000 0.06737045865954956
602080000 0.06642265549335877
602090000 0.06978558774493801
602100000 0.0625792758474946
602110000 0.06174913931977086
best so far: 0
type: [4, 1, 1, 27, 1] 108
cases of this type: 19683
602120000 0.04595573123061657
602130000 0.03672301230216027
best so far: 0
type: [4, 1, 3, 1, 9] 108
cases of this type: 59049
602140000 0.06386377813453144
602150000 0.061514412467373734
602160000 0.038266276152074334
602170000 0.039000131582187285
602180000 0.04271594330173069
602190000 0.07383872781423728
best so far: 0
type: [4, 1, 3, 3, 3] 108
cases of this type: 19683
602200000 0.05154954283618265
602210000 0.044721638534055815
best so far: 0
type: [4, 1, 3, 9, 1] 108
cases of this type: 6561
best so far: 0
type: [4, 1, 9, 1, 3] 108
cases of this type: 6

best so far: 0
type: [9, 6, 2, 1, 1] 108
cases of this type: 24
best so far: 0
type: [9, 12, 1, 1, 1] 108
cases of this type: 12
best so far: 0
type: [12, 1, 1, 1, 9] 108
cases of this type: 6561
best so far: 0
type: [12, 1, 1, 3, 3] 108
cases of this type: 2187
602670000 0.0006102217642664909
best so far: 0
type: [12, 1, 1, 9, 1] 108
cases of this type: 729
best so far: 0
type: [12, 1, 3, 1, 3] 108
cases of this type: 729
best so far: 0
type: [12, 1, 3, 3, 1] 108
cases of this type: 243
best so far: 0
type: [12, 1, 9, 1, 1] 108
cases of this type: 81
best so far: 0
type: [12, 3, 1, 1, 3] 108
cases of this type: 243
best so far: 0
type: [12, 3, 1, 3, 1] 108
cases of this type: 81
best so far: 0
type: [12, 3, 3, 1, 1] 108
cases of this type: 27
best so far: 0
type: [12, 9, 1, 1, 1] 108
cases of this type: 9
best so far: 0
type: [18, 1, 1, 1, 6] 108
cases of this type: 1296
best so far: 0
type: [18, 1, 1, 2, 3] 108
cases of this type: 648
best so far: 0
type: [18, 1, 1, 3, 2] 108
cases o